This folder collects the data given the csv. 

Load requisite libraries

In [3]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd 
# from sklearn.cluster import KMeans
# import matplotlib.pyplot as plt
import numpy as np
from requests.exceptions import RequestException


Build methods for cpaturing relevant html content

In [4]:
#methods
#total number of words in page     
def total_Words(soup):
    total_words = 0
    paragraphs = soup.find_all("p")
    for p in paragraphs: 
        total_words += len(p.get_text().split())
    return total_words
    
    
def total_Characters(soup): 
    total_char = 0
    paragraph = soup.find_all("p")
    for p in paragraph: 
        total_char += len(p.get_text())
    return total_char
    
    
#total number of references 
def total_Ref(soup): 
    references_section = soup.find('ol', class_= 'references')

    if references_section:
        references = references_section.find_all('li')
        num_references = len(references)
       # print(f"Number of references: {num_references}")
        return num_references
    else:
        #print("No references found.")    
        return 0
    
#end of methods

Load data and create a subset for testing

In [25]:
birdSpecies = pd.read_csv("../data/speciesList.csv")

testSubset = birdSpecies.iloc[1435:1445, 1]

print(testSubset)

1435               Pipreola jucunda
1436           Pipreola lubomirskii
1437               Pipreola pulchra
1438             Pipreola riefferii
1439          Pipreola squamipectus
1440              Pipreola whitelyi
1441    Porphyrolaema porphyrolaema
1442                 Procnias albus
1443               Procnias averano
1444            Procnias nudicollis
Name: Species1, dtype: object


Build the scrape. Start by creating lists of the right length, then save the base url as a variable. After that, use a for loop to first check the status code of a given species; if it's 200, proceed as usual, if it's not, upate 'Error' to hold the non 200 status code. This is actually a nice example, as it shows that if a species does not return a page, it actually produces a 404 error, so that's one less check that you need to do.

In [30]:
# create empty lists of the appropriate size for each variable being captured
NumReferences = [None] * len(testSubset)
NumChar = [None] * len(testSubset)
NumWords = [None] * len(testSubset)
Error = [None] * len(testSubset)

wikipedia_base = "https://en.wikipedia.org/wiki/"

for i in range(len(testSubset)):
    url = wikipedia_base + testSubset.iloc[i]
    response = rq.get(url)
    if response.status_code == 200:
        Error[i] = response.status_code
        html_doc = response.text
        # in here you now handle the html code
    else:
        Error[i] = response.status_code

print(Error)
print(testSubset)

[200, 200, 200, 200, 404, 200, 200, 200, 200, 200]
1435               Pipreola jucunda
1436           Pipreola lubomirskii
1437               Pipreola pulchra
1438             Pipreola riefferii
1439          Pipreola squamipectus
1440              Pipreola whitelyi
1441    Porphyrolaema porphyrolaema
1442                 Procnias albus
1443               Procnias averano
1444            Procnias nudicollis
Name: Species1, dtype: object


In [ ]:
# merge your lists into a df

In [86]:



for x in testSubset: 
    
    try:     
        species = x
        print(species)
        wikipedia_base = "https://en.wikipedia.org/wiki/"
        article = wikipedia_base + species
        html_doc = rq.get(article).text
        response = rq.get(article)
        errorBoolean = True
        
        soup = bs(html_doc, 'html.parser')
        
        
        if response.status_code != 200: 
            errorBoolean = False
            print ("page not successful")
            continue

        a = total_Characters(soup)
        b = total_Words(soup)
        c = total_Ref(soup)

        new_row = pd.DataFrame({"SpeciesName": [species], "NumReferences": [c], "NumChar": [a], "NumWords": [b], "Error": 
                               [errorBoolean]})

        df = pd.concat([df, new_row], ignore_index=True)


    except Exception as e: 
        print(f"An error occurred for {species}: {e}")
        
        
print(df)


Pipreola jucunda
Pipreola lubomirskii
Pipreola pulchra
Pipreola riefferii
Pipreola squamipectus
page not successful
Pipreola whitelyi
Porphyrolaema porphyrolaema
Procnias albus
Procnias averano
Procnias nudicollis
                    SpeciesName NumReferences NumChar NumWords  Error
0              Pipreola jucunda             3    2142      346    NaN
1          Pipreola lubomirskii             3    1854      299    NaN
2              Pipreola pulchra             3    1607      270    NaN
3            Pipreola riefferii             3    1653      274    NaN
4             Pipreola whitelyi             3    2343      380    NaN
5   Porphyrolaema porphyrolaema            12    7519     1184    NaN
6                Procnias albus             6    1905      319    NaN
7              Procnias averano             6    3977      645    NaN
8           Procnias nudicollis             7    5634      882    NaN
9              Pipreola jucunda             3    2142      346  False
10         Pipre

In [89]:
#random cell to test

0.6394267984578837


In [88]:
print(soup.title.string) 
print(soup.find("h2", {"id": "References"})) #if it doesn't find anything, it will return none. If it does find it, it will return tag (h2) and the text 
#h2Ref = (soup.find_all("h2"))
#print(h2Ref)
#type(h2Ref)
#h2Ref.tag[id]

#looks to see if there is an h2 with the id references. If there is, do [X], and if not, do [Y]. 
#swap out whatever criteria Sandra deems sufficient evidence 
#refences is more guaranteed to exist as a header
#extract number of words in the wikipedia page, ... is that sufficient evidence??
#list of all species 
#construct a list of boolean values 
#if eveidcene exist, 
# Bird1 - True
# Bird2- False 
#Bird3 - True 

#use pandas to create a data frame 
#combining those 2 lists to create a rectangular dataset and create a csv 
# 2 variables: name, boolean value (T or F)

Bare-throated bellbird - Wikipedia
<h2 id="References">References</h2>


In [4]:
#everyhting we have dones so far assume wikipedia entries ... what if it does not exist
#find a weak speceis and look to see what returns 
#unique identify of no article exists 
# then put a false 
#What to do if species dne on wikipedia 

#assuming we successfully connected to wikipedia and there is no server error 

# number 1 thing to do is verfiy there is no server error!! 
# request library that talks to the server allows to figure out what type of error 
# server number is 200: which means it works 
#anything else is invalid 
# a list of server response codes 
#request if response code is 200 
#look into species 
#if response code is not 200, record a false in the boolean list 

#do a few for a test 
#what are limits to scraping wikipedia ... look up on google 
#what is the number per second, per day, per week...


In [11]:
import pandas as pd

df = pd.DataFrame(columns=["SpeciesName", "NumReferences", "NumParagraphs", "NumWords", "Error"])



birdSpecies = ["Accipiter albogularis", "Accipiter badius", "Accipiter bicolor"]

    
for i in birdSpecies:
    # Create a DataFrame for the new row
    new_row = pd.DataFrame({"SpeciesName": [i], "NumReferences": [num_references], "NumParagraphs": [totalParagraph], "NumWords": [totalWords]})
    
    # Append the new row to the existing DataFrame
    df = pd.concat([df, new_row], ignore_index=True)

# Display the DataFrame
print(df)

['Accipiter albogularis', 'Accipiter badius', 'Accipiter bicolor']
Accipiter albogularis
Accipiter badius
Accipiter bicolor
             SpeciesName NumReferences NumParagraphs NumWords Error
0  Accipiter albogularis            35            12     6188   NaN
1       Accipiter badius            35            12     6188   NaN
2      Accipiter bicolor            35            12     6188   NaN
